# Map Consensus (Experimental Feature)

## Overview

This document describes an **experimental feature** called **Map Consensus**, which explores the idea of **distributed map fusion through neighbor communication**.

The current implementation is a **proof-of-concept (PoC)** intended to test the `combine_maps` mechanism, where each node iteratively fuses its local map with maps received from neighboring nodes in a predefined communication graph.

This idea may be useful in the future for **multi-robot mapping or cooperative SLAM**, but at present it remains an exploratory concept.

---

In [28]:
from multiprocessing import Pool


def graph_server() -> None:
    from logging import basicConfig, INFO

    basicConfig(level=INFO)

    from topolink import Graph

    nodes = ["1", "2", "3", "4", "5"]
    edges = [("1", "2"), ("2", "3"), ("3", "4"), ("4", "5"), ("5", "1")]

    graph = Graph(nodes, edges)

    graph.deploy()


def map_consensus(idx: str, map: dict[str, float], n_iter: int = 100) -> None:
    import logging
    from topolink import NodeHandle

    logging.basicConfig(level=logging.INFO)

    nh = NodeHandle(idx)

    alpha = 0.5

    for k in range(n_iter):
        map = nh.combine_maps(map, alpha)
        logging.info(f"Node {idx} - Iteration {k + 1}: {map}")


if __name__ == "__main__":
    N_NODES = 5

    map_for_all = {
        "1": {"[0, 0]": 0.5, "[0, 1]": 0.5, "[1, 0]": 0.0, "[1, 1]": 0.0},
        "2": {"[0, 2]": 0.3, "[0, 3]": 0.7, "[1, 2]": -2.0, "[1, 3]": -1.0},
        "3": {"[2, 0]": 0.2, "[2, 1]": 0.8, "[3, 0]": 2.0, "[3, 1]": 3.0},
        "4": {"[2, 2]": 0.6, "[3, 2]": 0.4, "[2, 3]": 1.0, "[3, 3]": 2.0},
        "5": {"[1, 1]": 0.9, "[1, 2]": 0.1, "[2, 1]": -1.0, "[2, 2]": -0.5},
    }

    with Pool(N_NODES + 1) as pool:
        server = pool.apply_async(graph_server)
        tasks = [
            pool.apply_async(
                map_consensus, args=(str(i + 1), map_for_all.get(str(i + 1), {}))
            )
            for i in range(N_NODES)
        ]
        node_states = [task.get() for task in tasks]
        server.get()

INFO:topolink.graph:Graph 'default' running on: 192.168.1.102:44637
INFO:topolink.discovery:Graph service 'default._topolink._tcp.local.' added
INFO:topolink.discovery:Graph service 'default._topolink._tcp.local.' added
INFO:topolink.discovery:Graph service 'default._topolink._tcp.local.' added
INFO:topolink.discovery:Graph service 'default._topolink._tcp.local.' added
INFO:topolink.discovery:Graph service 'default._topolink._tcp.local.' added
INFO:topolink.discovery:Registered graph service with name 'default'
INFO:topolink.graph:Node '1' joined graph 'default' from 192.168.1.102:45211.
INFO:topolink.graph:Node '4' joined graph 'default' from 192.168.1.102:45459.
INFO:topolink.node_handle:Node '1' joined graph 'default'.
INFO:topolink.graph:Node '2' joined graph 'default' from 192.168.1.102:46619.
INFO:topolink.node_handle:Node '4' joined graph 'default'.
INFO:topolink.node_handle:Node '2' joined graph 'default'.
INFO:topolink.graph:Node '3' joined graph 'default' from 192.168.1.102:4